In [1]:
import openai
import pandas as pd
import logging
import time
from tqdm import tqdm

# Set up logging
logging.basicConfig(filename='translation_errors_eng.log', level=logging.ERROR)

# Load your API key from an environment variable or directly set it
openai.api_key = 'PUT API KEY HERE'

# Load your dataframe
df = pd.read_csv('flowers.csv') 
df2 = df.sample(n=10000, random_state=1)  # Replace with your actual file path
historical_sentences = df2['pre_text'].tolist()

# Function to translate historical Dutch to modern Dutch using GPT-3.5-turbo
def translate_historical_to_modern(sentence):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Using GPT-3.5-turbo model
            messages=[
                {"role": "system", "content": "Translate the by the user provided historical English text to modern English. Return only the modern English text and nothing else."},
                {"role": "user", "content": f"{sentence}"}
            ]
        )
        translated_sentence = response.choices[0].message['content'].strip()
        return translated_sentence
    except Exception as e:
        logging.error(f"Error translating sentence: {sentence}\nException: {e}")
        return None

# Measure the total translation time
start_time = time.time()

# Translate all historical Dutch sentences to modern Dutch with progress indication
modern_sentences = []
for sentence in tqdm(historical_sentences, desc="Translating sentences"):
    translated_sentence = translate_historical_to_modern(sentence)
    modern_sentences.append(translated_sentence)

df2['modern_sentences'] = modern_sentences

# Measure the end time
end_time = time.time()
total_time = end_time - start_time

# Log the total time taken
logging.info(f"Total time taken for translation: {total_time} seconds")

# Save the translated sentences, including those with None
df2.to_csv('translated_sentences_10k_EN.csv', index=False)

print(f"Total time taken for translation: {total_time / 60:.2f} minutes")


/Users/patrickjonathan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
Translating sentences: 100%|██████████| 10000/10000 [6:10:38<00:00,  2.22s/it]  


Total time taken for translation: 370.65 minutes
